# Extra notebook - Exporting graphs and data for external visualization, further calculations
## Project: Growing Urban Bicycle Networks

This is a temporary notebook which was needed to export data for creating the growbike.net platform. It takes the produced graphs or data from previous steps and exports the data in a format in which it can be used in an external visualization, or for further calculations. 

Contact: Michael Szell (michael.szell@gmail.com)  
Created: 2020-09-25  
Last modified: 2021-06-26

## Preliminaries

### Parameters

In [8]:
debug = False # If True, will produce plots and/or verbose output to double-check
%run -i "../parameters/parameters.py"

Loaded parameters.



### Setup

In [9]:
%run -i path.py
%run -i setup.py

%load_ext watermark
%watermark -n -v -m -g -iv

Loaded PATH.

Setup finished.

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
Python implementation: CPython
Python version       : 3.12.6
IPython version      : 8.27.0

Compiler    : MSC v.1941 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 140 Stepping 1, GenuineIntel
CPU cores   : 8
Architecture: 64bit

Git hash: 19e8d1dcd8a92936e90ccacb37142aa1f2c65fdd

IPython   : 8.27.0
matplotlib: 3.8.4
igraph    : 0.11.6
csv       : 1.0
geojson   : 3.1.0
networkx  : 3.3
numpy     : 1.26.4
shapely   : 2.0.6
geopandas : 0.14.4
watermark : 2.5.0
osmnx     : 1.9.4
haversine : 2.8.1
json      : 2.0.9
pandas    : 2.2.3
tqdm      : 4.66.5
pyproj    : 3.6.1
osgeo     : 3.9.2
fiona     : 1.10.1
sys       : 3.12.6 | packaged by conda-forge | (main, Sep 22 2024, 14:01:26) [MSC v.1941 64 bit (AMD64)]



### Functions

In [10]:
%run -i functions.py

Loaded functions.



In [11]:
# Run all parameter sets
poi_source_list = [
                    "grid", 
                   "railwaystation", 
                   "neighbourhoods"
                   ]
prune_measure_list = ["betweenness", "closeness", "random"]
combs = list(itertools.product(poi_source_list, prune_measure_list))

## Igraph to GeoJSON

In [12]:
# edited to avoid stopping if all types of parameters haven't been run yet - 09 Oct 2024

In [13]:
for placeid, placeinfo in tqdm(cities.items(), desc="Cities"):
    print(placeid + ": Exporting streets to GeoJSON")
    Gs = {}
    for networktype in networktypes:
        Gs[networktype] = csv_to_ig(PATH["data"] + placeid + "/", placeid, networktype, weighting)
        Gs[networktype + "_simplified"] = csv_to_ig(PATH["data"] + placeid + "/", placeid, networktype + "_simplified", weighting)
    for nw, G in Gs.items():
        G_geojson = ig_to_geojson(G)
        # Only add '_weighted' if it's not already present in the networktype
        filename_suffix = "_weighted" if weighting and "weighted" not in nw else ""
        with open(PATH["exports_json"] + placeid + "/" + placeid + "_" + nw + filename_suffix + '.json', 'w') as f:
            geojson.dump(G_geojson, f)
    print(placeid + ": Exporting simulation results to GeoJSON")
    for poi_source, prune_measure in combs:
        # Modify filename based on weighting flag
        weighting_str = "_weighted" if weighting else ""
        filename = placeid + '_poi_' + poi_source + "_" + prune_measure + weighting_str
        try:
            # Try loading the file with or without the "_weighted" suffix
            with open(PATH["results"] + placeid + "/" + filename + ".pickle", 'rb') as resultfile:
                res = pickle.load(resultfile)
            if debug:
                pp.pprint(res)

            # Process the results
            for GT, GT_abstract, prune_quantile in zip(res["GTs"], res["GT_abstracts"], res["prune_quantiles"]):
                GT_geojson = ig_to_geojson(GT)
                
                # Add '_weighted' only if it's not already part of the filename
                filename_suffix = "_weighted" if weighting and "weighted" not in prune_measures[prune_measure] else ""
                with open(PATH["exports_json"] + placeid + "/" + placeid + '_GTbonly_poi_' + poi_source + "_" + prune_measures[prune_measure] + "{:.3f}".format(prune_quantile) + filename_suffix + '.json', 'w') as f:
                    geojson.dump(GT_geojson, f)

        except FileNotFoundError:
            # Attempt to load the file without the weighting suffix if it fails
            if weighting:
                filename = placeid + '_poi_' + poi_source + "_" + prune_measure
                try:
                    with open(PATH["results"] + placeid + "/" + filename + ".pickle", 'rb') as resultfile:
                        res = pickle.load(resultfile)
                    # Successfully loaded the file without weighting, process as normal
                    for GT, GT_abstract, prune_quantile in zip(res["GTs"], res["GT_abstracts"], res["prune_quantiles"]):
                        GT_geojson = ig_to_geojson(GT)
                        with open(PATH["exports_json"] + placeid + "/" + placeid + '_GTbonly_poi_' + poi_source + "_" + prune_measures[prune_measure] + "{:.3f}".format(prune_quantile) + '.json', 'w') as f:
                            geojson.dump(GT_geojson, f)
                except FileNotFoundError:
                    print(f"File not found for {filename} even after removing '_weighted', skipping to the next.")
            else:
                print(f"File not found for {filename}, skipping to the next.")


Cities:   0%|          | 0/1 [00:00<?, ?it/s]

newcastle: Exporting streets to GeoJSON


C:\Users\b8008458\OneDrive - Newcastle University\2022 to 2023\PhD\networkGrowth\bikenwgrowth\code\functions.py:448: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  e = pd.read_csv(p + prefix + '_edges.csv')


newcastle: Exporting simulation results to GeoJSON
File not found for newcastle_poi_grid_closeness even after removing '_weighted', skipping to the next.
File not found for newcastle_poi_grid_random even after removing '_weighted', skipping to the next.
File not found for newcastle_poi_railwaystation_betweenness even after removing '_weighted', skipping to the next.
File not found for newcastle_poi_railwaystation_closeness even after removing '_weighted', skipping to the next.
File not found for newcastle_poi_railwaystation_random even after removing '_weighted', skipping to the next.
File not found for newcastle_poi_neighbourhoods_closeness even after removing '_weighted', skipping to the next.
File not found for newcastle_poi_neighbourhoods_random even after removing '_weighted', skipping to the next.


In [14]:
## convert json outputs to geopackage to load into QGIS easily
# Ensure output directory exists
PATH['exports_gpkg'] = '../../bikenwgrowth_external/exports_gpkg/'
output_directory = os.path.abspath(PATH['exports_gpkg'])
os.makedirs(output_directory, exist_ok=True)

# Function to process each JSON file
def process_json_file(input_file, placeid):
    with open(input_file) as f:
        data = json.load(f)

    # Check if the data is a GeometryCollection
    if data["type"] == "GeometryCollection":
        # Extract only LineString geometries and create LineString objects
        lines = [
            LineString(g["coordinates"]) for g in data["geometries"] if g["type"] == "LineString"
        ]

        # Create a GeoDataFrame from the LineStrings
        if lines:
            gdf_lines = gpd.GeoDataFrame(geometry=lines, crs="EPSG:4326")
            # Ensure placeid-specific output directory exists
            place_output_directory = os.path.join(output_directory, placeid)
            os.makedirs(place_output_directory, exist_ok=True)
            # Save the GeoDataFrame to a GeoPackage
            output_file_gpkg = os.path.join(place_output_directory, os.path.basename(input_file).replace(".json", ".gpkg"))
            gdf_lines.to_file(output_file_gpkg, driver='GPKG', layer='lines')

            if debug:
                print(f"Converted {os.path.basename(input_file)} to GeoPackage and saved to {place_output_directory}")

# Function to process all JSON files in a directory
def process_all_json_files(base_directory):
    for root, dirs, files in os.walk(base_directory):
        for file in files:
            if file.endswith('.json'):
                input_file = os.path.join(root, file)
                placeid = os.path.basename(root)  # Using the name of the folder as placeid
                if debug == True:
                    print(f"Processing file: {input_file} for placeid: {placeid}")
                process_json_file(input_file, placeid)

# Specify the base directory where your JSON files are stored
base_directory = PATH["exports_json"]  # Adjust this to your base directory

print("Starting conversion of all JSON files to GeoPackages...")
process_all_json_files(base_directory)
print("All conversions completed.")

Starting conversion of all JSON files to GeoPackages...
All conversions completed.


## Igraph picklez of simplified carconstrictedbike networks

In [ ]:
for placeid, placeinfo in tqdm(cities.items(), desc = "Cities"):
    print(placeid + ": Exporting carconstrictedbike to picklez")
    
    # Load existing
    G_carall = csv_to_ig(PATH["data"] + placeid + "/", placeid, 'carall')
    with open(PATH["exports"] + placeid + "/" + placeid + '_carall.picklez', 'wb') as f:
        G_carall_simplified = simplify_ig(G_carall)
        G_carall_simplified.write_picklez(fname = f)
    if debug: map_center = nxdraw(G_carall, "carall")
            
    # Load results
    filename = placeid + '_poi_' + poi_source + "_" + prune_measure
    resultfile = open(PATH["results"] + placeid + "/" + filename + ".pickle",'rb')
    res = pickle.load(resultfile)
    resultfile.close()
    
    if debug:
        fig = initplot()
        nxdraw(G_carall_simplified, "abstract", map_center, nodesize = 0, weighted = True, maxwidthsquared = 500/100)
        plt.savefig(PATH["exports"] + placeid + "/" + placeid + '_carallweighted.png', bbox_inches="tight", dpi=plotparam["dpi"])
        plt.close()
    for GT, prune_quantile in zip(res["GTs"], tqdm(res["prune_quantiles"], desc = "Growth stages", leave = False)):
        if prune_quantile in prune_quantiles: #[0.5,1]:
            GT_carconstrictedbike = copy.deepcopy(G_carall)
            constrict_overlaps(GT_carconstrictedbike, GT)
            GT_carconstrictedbike = simplify_ig(GT_carconstrictedbike)
            if debug:
                fig = initplot()
                nxdraw(GT_carconstrictedbike, "abstract", map_center, nodesize = 0, weighted = True, maxwidthsquared = 500)
                plt.savefig(PATH["exports"] + placeid + "/" + placeid + '_carconstrictedbike_poi_' + poi_source + "_" + prune_measures[prune_measure] + "{:.3f}".format(prune_quantile) + '.png', bbox_inches="tight", dpi=plotparam["dpi"])
                plt.close()
            with open(PATH["exports"] + placeid + "/" + placeid + '_carconstrictedbike_poi_' + poi_source + "_" + prune_measures[prune_measure] + "{:.3f}".format(prune_quantile) + '.picklez', 'wb') as f:
                GT_carconstrictedbike.write_picklez(fname = f)

## Railwaystation POIs gdf to GeoJSON

In [ ]:
for placeid, placeinfo in tqdm(cities.items(), desc = "Cities"):
    try:
        poi_source_here = "railwaystation"
        poi_gdf = gpd.GeoDataFrame.from_file(PATH["data"] + placeid + "/" + placeid + '_' + 'poi_' + poi_source_here + '.gpkg')
    #         pp.pprint(poi_gdf.geometry.x)
        with open(PATH["exports_json"] + placeid + "/" + placeid + '_poi_' + poi_source_here + '.json', 'w') as f:
            geojson.dump(gdf_to_geojson(poi_gdf, poi_gdf.keys()), f)
    except:
        print(placeid + " did not work.")

## Grid and railwaystation POIs to lat/lon list

Given [placeid]_poi_grid_nnidscarall, fetch and export x,y (lat,lon) from [placeid]_carall_nodes.csv to [placeid]_poi_grid_latlon.csv.

In [ ]:
for placeid, placeinfo in tqdm(cities.items(), desc = "Cities"):
    if check_extract_zip(PATH["data"] + placeid + "/", placeid + "_carall"):
        for poi_source_here in ["railwaystation", "grid"]:
            with open(PATH["data"] + placeid + "/" + placeid + '_poi_' + poi_source_here + '_nnidscarall.csv', 'r') as fin, open(PATH["exports_json"] + placeid + "/" + placeid + '_poi_' + poi_source_here + '_latlon.csv', 'w') as fout:
            
                fdata = np.genfromtxt(PATH["data"] + placeid + "/" + placeid + '_carall_nodes.csv', delimiter=',', usecols=(0,1,2))
                for line in fin:
                    poiid = int(line.strip())
                    fdata_lineid = np.argwhere(fdata[:, 2] == poiid)
                    fout.write(str(fdata[fdata_lineid, 0].flatten()[0]) + "," + str(fdata[fdata_lineid, 1].flatten()[0]) + '\n')

        os.remove(PATH["data"] + placeid + "/" + placeid + "_carall_nodes.csv")
        os.remove(PATH["data"] + placeid + "/" + placeid + "_carall_edges.csv")

## GeoJSON Linter

https://geojsonlint.com/

## Copy selected videos, plots, and results

In [ ]:
copyfiles = {"plots": 
                ["_analysis_poi_grid_betweenness.png",
                 "_analysis_poi_grid_closeness.png",
                 "_analysis_poi_grid_random.png",
                 "_analysis_poi_railwaystation_betweenness.png",
                 "_analysis_poi_railwaystation_closeness.png",
                 "_analysis_poi_railwaystation_random.png"
                ],
             "plots_networks":
                ["_biketrack.pdf",
                 "_bikeable.pdf",
                 "_biketrackcarall.pdf",
                 "_carall.pdf",
                 "_carall_poi_grid.pdf",
                 "_carall_poi_railwaystation.pdf",
                 "_MSTabstract_poi_grid.pdf",
                 "_MSTabstract_poi_railwaystation.pdf",
                 "_MSTall_poi_grid.pdf",
                 "_MSTall_poi_railwaystation.pdf",
                 "_GTallcover_poi_grid_Bq1.000.png",
                 "_GTallcover_poi_railwaystation_Bq1.000.png"
                ],
            "videos":
                ["_GTabstract_poi_grid_Bq.mp4",
                 "_GTabstract_poi_grid_Cq.mp4",
                 "_GTabstract_poi_grid_Rq.mp4",
                 "_GTabstract_poi_railwaystation_Bq.mp4",
                 "_GTabstract_poi_railwaystation_Cq.mp4",
                 "_GTabstract_poi_railwaystation_Rq.mp4",
                 "_GTall_poi_grid_Bq.mp4",
                 "_GTall_poi_grid_Cq.mp4",
                 "_GTall_poi_grid_Rq.mp4",
                 "_GTall_poi_railwaystation_Bq.mp4",
                 "_GTall_poi_railwaystation_Cq.mp4",
                 "_GTall_poi_railwaystation_Rq.mp4",
                 "_GTallcover_poi_grid_Bq.mp4",
                 "_GTallcover_poi_grid_Cq.mp4",
                 "_GTallcover_poi_grid_Rq.mp4",
                 "_GTallcover_poi_railwaystation_Bq.mp4",
                 "_GTallcover_poi_railwaystation_Cq.mp4",
                 "_GTallcover_poi_railwaystation_Rq.mp4",
                 "_GTabstract_poi_grid_Bq.webm",
                 "_GTabstract_poi_grid_Cq.webm",
                 "_GTabstract_poi_grid_Rq.webm",
                 "_GTabstract_poi_railwaystation_Bq.webm",
                 "_GTabstract_poi_railwaystation_Cq.webm",
                 "_GTabstract_poi_railwaystation_Rq.webm",
                 "_GTall_poi_grid_Bq.webm",
                 "_GTall_poi_grid_Cq.webm",
                 "_GTall_poi_grid_Rq.webm",
                 "_GTall_poi_railwaystation_Bq.webm",
                 "_GTall_poi_railwaystation_Cq.webm",
                 "_GTall_poi_railwaystation_Rq.webm",
                 "_GTallcover_poi_grid_Bq.webm",
                 "_GTallcover_poi_grid_Cq.webm",
                 "_GTallcover_poi_grid_Rq.webm",
                 "_GTallcover_poi_railwaystation_Bq.webm",
                 "_GTallcover_poi_railwaystation_Cq.webm",
                 "_GTallcover_poi_railwaystation_Rq.webm"
                  ],
             "results":
                ["_existing.csv",
                 "_poi_railwaystation_random.csv",
                 "_poi_grid_random.csv",
                 "_poi_railwaystation_closeness.csv",
                 "_poi_grid_closeness.csv",
                 "_poi_railwaystation_betweenness.csv",
                 "_poi_grid_betweenness.csv"
                ]
            }

In [ ]:
exportpath = PATH["exports_json"] #"/Users/misz/Dropbox/supervision/2021ceciliamorten_bikeviz/data/" 
for placeid, placeinfo in tqdm(cities.items(), desc = "Cities"):
    for k, v in copyfiles.items():
        for filename in v:
            try:
                shutil.copy2(PATH[k] + placeid + "/" + placeid + filename, exportpath + placeid + "/")
            except:
                print("File not found: " + placeid + filename)